In [1]:
from tiatoolbox.wsicore.wsireader import OpenSlideWSIReader, WSIReader, TIFFWSIReader
from tiatoolbox.models.dataset.classification import WSIPatchDataset
from pprint import pprint
import tifffile 
from tiatoolbox.tools import patchextraction
#from tiatoolbox.tools.patchextraction import get_coordinates
import numpy as np
import palom
import skimage.transform
import skimage.exposure
from tiatoolbox import logger
from tiatoolbox.models.engine.patch_predictor import (
    IOPatchPredictorConfig,
    PatchPredictor,)
import os 
import pickle
import skimage.exposure
import torchvision.transforms
from tiatoolbox import data
from tiatoolbox.tools import stainnorm
import cv2

|2024-08-21|13:58:23.806| [WARNING] /home/evos/miniconda/envs/tiatoolbox/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



In [11]:
def preproc_func_gamma(img):
    return torchvision.transforms.ToTensor()(
        skimage.exposure.adjust_gamma(img, 2.2)
    ).permute(1, 2, 0)


def preproc_func_macenko(img):
    target_image_dir = "/home/evos/Data/COLO/normalization_template.jpg"  #SET CORRECT TARGET IMAGE PATH
    target_image = cv2.imread(target_image_dir)
    stain_normalizer = stainnorm.get_normalizer('Macenko')
    stain_normalizer.fit(target_image)
    normalized_img = stain_normalizer.transform(img.copy())

    return normalized_img


In [12]:
def predict_orion_kather(tiles_dir, slides_dir, output_dir, preproc_func, pretrained_model, batch_size=32):

    slides = os.listdir(slides_dir)
    counter=1
    for slide_name in os.listdir(slides_dir):
        print(counter)
        patch_list = []
        coordinates_ordered = np.empty((0, 2), dtype=np.int64)
        tiles_dir = "/home/evos/Outputs/COLO/Orion_tiles"
        for tile_name in os.listdir(tiles_dir):
            if slide_name in tile_name:
                full_path = os.path.join(tiles_dir, tile_name)
                patch_list.append(full_path)
                tile_name = tile_name.split(".")[0]
                x_coor = int(tile_name.split("_")[-2])
                y_coor = int(tile_name.split("_")[-1])
                coor = np.array([x_coor, y_coor])
                coordinates_ordered = np.vstack([coordinates_ordered, coor])

        ON_GPU = True

        pretrained_model=pretrained_model
        predictor = PatchPredictor(pretrained_model=pretrained_model, batch_size=batch_size)
        
        if preproc_func == 'gamma':
            predictor.model.preproc_func = preproc_func_gamma

        output = predictor.predict(imgs=patch_list, mode="patch", on_gpu=ON_GPU, return_probabilities=True)

        output["coordinates"]=coordinates_ordered

        preproc_func = 'gamma'
        predictions_file_path = "{}/{}_{}_{}_cell_type_map.pkl".format(output_dir, slide_name, preproc_func, pretrained_model)
        print(predictions_file_path)

        with open(predictions_file_path, 'wb') as f:
            pickle.dump(output, f)

        counter=counter+1

    print("Finished making Orion predictions")

In [14]:
slide_name = "18459_LSP10353_US_SCAN_OR_001__093059-registered"
patch_list = []
coordinates_ordered = np.empty((0, 2), dtype=np.int64)
tiles_dir = "/home/evos/Outputs/COLO/Orion_tiles"
for tile_name in os.listdir(tiles_dir):
    if slide_name in tile_name:
        full_path = os.path.join(tiles_dir, tile_name)
        patch_list.append(full_path)
        tile_name = tile_name.split(".")[0]
        x_coor = int(tile_name.split("_")[-2])
        y_coor = int(tile_name.split("_")[-1])
        coor = np.array([x_coor, y_coor])
        coordinates_ordered = np.vstack([coordinates_ordered, coor])

print(len(patch_list))

10446


In [16]:
output_dir = "/home/evos/Outputs/COLO/Orion_DL_predictions"
slides_dir = "/home/evos/Data/COLO/Orion_slides"
target_image_dir = "/home/evos/Data/COLO/normalization_template.jpg"

ON_GPU = True

pretrained_model='densenet161-kather100k'
#pretrained_model='resnet18-kather100k'
batch_size=32

predictor = PatchPredictor(pretrained_model=pretrained_model, batch_size=32)
predictor.model.preproc_func = preproc_func_macenko

output = predictor.predict(imgs=patch_list, mode="patch", on_gpu=ON_GPU, return_probabilities=True)

output["coordinates"]=coordinates_ordered

preproc_func = "Macenko"
predictions_file_path = "{}/{}_{}_{}_cell_type_map.pkl".format(output_dir, slide_name, preproc_func, pretrained_model)
print(predictions_file_path)

with open(predictions_file_path, 'wb') as f:
    pickle.dump(output, f)


100%|#########################################| 327/327 [17:03<00:00,  3.13s/it]

/home/evos/Outputs/COLO/Orion_DL_predictions/18459_LSP10353_US_SCAN_OR_001__093059-registered_Macenko_densenet161-kather100k_cell_type_map.pkl
